In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip "/content/drive/My Drive/ti" -d "/content/ti"
!unzip "/content/drive/My Drive/to" -d "/content/to"


Streaming output truncated to the last 5000 lines.
  inflating: /content/to/to/ot1439.jpg  
  inflating: /content/to/to/ot4685.jpg  
  inflating: /content/to/to/ot5680.jpg  
  inflating: /content/to/to/ot2908.jpg  
  inflating: /content/to/to/ot4937.jpg  
  inflating: /content/to/to/ot938.jpg  
  inflating: /content/to/to/ot6208.jpg  
  inflating: /content/to/to/ot4429.jpg  
  inflating: /content/to/to/ot1044.jpg  
  inflating: /content/to/to/ot2336.jpg  
  inflating: /content/to/to/ot953.jpg  
  inflating: /content/to/to/ot4444.jpg  
  inflating: /content/to/to/ot61.jpg  
  inflating: /content/to/to/ot2932.jpg  
  inflating: /content/to/to/ot5558.jpg  
  inflating: /content/to/to/ot1454.jpg  
  inflating: /content/to/to/ot4187.jpg  
  inflating: /content/to/to/ot4365.jpg  
  inflating: /content/to/to/ot5647.jpg  
  inflating: /content/to/to/ot5843.jpg  
  inflating: /content/to/to/ot5922.jpg  
  inflating: /content/to/to/ot1276.jpg  
  inflating: /content/to/to/ot5665.jpg  
  inflatin

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255)

train_generator = datagen.flow_from_directory(
        '/content',
        target_size=(256, 256),
        batch_size=16,
        class_mode=None,
        subset='training',
        shuffle=True,
        seed=42)

test_generator = datagen.flow_from_directory(
        '/content',
        target_size=(256, 256),
        batch_size=16,
        class_mode=None,
        subset='validation',
        shuffle=True,
        seed=42)


Found 24425 images belonging to 5 classes.
Found 0 images belonging to 5 classes.


In [ ]:
# Generator
def build_generator():
    inputs = tf.keras.layers.Input(shape=(256, 256, 3))
    
    down1 = tf.keras.layers.Conv2D(64, 4, strides=2, padding='same', activation='relu')(inputs)
    down2 = tf.keras.layers.Conv2D(128, 4, strides=2, padding='same', activation='relu')(down1)
    down3 = tf.keras.layers.Conv2D(256, 4, strides=2, padding='same', activation='relu')(down2)
    down4 = tf.keras.layers.Conv2D(512, 4, strides=2, padding='same', activation='relu')(down3)
    down5 = tf.keras.layers.Conv2D(512, 4, strides=2, padding='same', activation='relu')(down4)
    down6 = tf.keras.layers.Conv2D(512, 4, strides=2, padding='same', activation='relu')(down5)
    down7 = tf.keras.layers.Conv2D(512, 4, strides=2, padding='same', activation='relu')(down6)
    down8 = tf.keras.layers.Conv2D(512, 4, strides=2, padding='same', activation='relu')(down7)
    
    up1 = tf.keras.layers.Conv2DTranspose(512, 4, strides=2, padding='same', activation='relu')(down8)
    up1 = tf.keras.layers.Concatenate()([up1, down7])
    up2 = tf.keras.layers.Conv2DTranspose(512, 4, strides=2, padding='same', activation='relu')(up1)
    up2 = tf.keras.layers.Concatenate()([up2, down6])
    up3 = tf.keras.layers.Conv2DTranspose(512, 4, strides=2, padding='same', activation='relu')(up2)
    up3 = tf.keras.layers.Concatenate()([up3, down5])
    up4 = tf.keras.layers.Conv2DTranspose(512, 4, strides=2, padding='same', activation='relu')(up3)
    up4 = tf.keras.layers.Concatenate()([up4, down4])
    up5 = tf.keras.layers.Conv2DTranspose(256, 4, strides=2, padding='same', activation='relu')(up4)
    up5 = tf.keras.layers.Concatenate()([up5, down3])
    up6 = tf.keras.layers.Conv2DTranspose(128, 4, strides=2, padding='same', activation='relu')(up5)
    up6 = tf.keras.layers.Concatenate()([up6, down2])
    up7 = tf.keras.layers.Conv2DTranspose(64, 4, strides=2, padding='same', activation='relu')(up6)
    up7 = tf.keras.layers.Concatenate()([up7, down1])

    outputs = tf.keras.layers.Conv2DTranspose(3, 4, strides=2, padding='same', activation='tanh')(up7)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    return model


In [ ]:
def build_discriminator():
    inputs = tf.keras.layers.Input(shape=(256, 256, 6))

    conv1 = tf.keras.layers.Conv2D(64, 4, strides=2, padding='same')(inputs)
    conv1 = tf.keras.layers.LeakyReLU(alpha=0.2)(conv1)
    conv2 = tf.keras.layers.Conv2D(128, 4, strides=2, padding='same')(conv1)
    conv2 = tf.keras.layers.LeakyReLU(alpha=0.2)(conv2)
    conv3 = tf.keras.layers.Conv2D(256, 4, strides=2, padding='same')(conv2)
    conv3 = tf.keras.layers.LeakyReLU(alpha=0.2)(conv3)
    conv4 = tf.keras.layers.Conv2D(512, 4, strides=2, padding='same')(conv3)
    conv4 = tf.keras.layers.LeakyReLU(alpha=0.2)(conv4)
    outputs = tf.keras.layers.Conv2D(1, 4, strides=1, padding='valid')(conv4)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    return model


In [ ]:
def generator_loss(fake_output, target):
    gan_loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)(tf.ones_like(fake_output), fake_output)
    l1_loss = tf.keras.losses.MeanAbsoluteError()(target, fake_output)
    total_loss = gan_loss + 100 * l1_loss
    return total_loss






In [ ]:
def discriminator_loss(real_output, fake_output):
    real_loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)(tf.ones_like(real_output), real_output)
    fake_loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

generator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

def train_step(input_image, target):
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        # Generate fake output
        fake_output = generator(input_image, training=True)
        # Concatenate input and fake output
        concat_input_fake = tf.keras.layers.Concatenate()([input_image, fake_output])
    
        # Concatenate input and target output
        concat_input_target = tf.keras.layers.Concatenate()([input_image, target])
        
        # Calculate discriminator output for fake and target outputs
        fake_discriminator_output = discriminator(concat_input_fake, training=True)
        target_discriminator_output = discriminator(concat_input_target, training=True)
        
        # Calculate generator and discriminator losses
        gen_loss = generator_loss(fake_discriminator_output, target)
        disc_loss = discriminator_loss(target_discriminator_output, fake_discriminator_output)
    
        # Calculate gradients and apply to optimizer
        generator_gradients = gen_tape.gradient(gen_loss, generator.trainable_variables)
        discriminator_gradients = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
        generator_optimizer.apply_gradients(zip(generator_gradients, generator.trainable_variables))
        discriminator_optimizer.apply_gradients(zip(discriminator_gradients, discriminator.trainable_variables))

    return gen_loss, disc_loss




In [ ]:
generator = build_generator()
discriminator = build_discriminator()

generator.compile(optimizer=generator_optimizer, loss=generator_loss)
discriminator.compile(optimizer=discriminator_optimizer, loss=discriminator_loss)

BATCH_SIZE = 1
EPOCHS = 50

In [ ]:
import zipfile
import glob
import cv2
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# Define hyperparameters
BATCH_SIZE = 1
EPOCHS = 50

def create_dataset():
    # Load input and target images
    ti_zip = zipfile.ZipFile('/content/drive/MyDrive/ti.zip')
    ti_dir = ti_zip.extractall()
    to_zip = zipfile.ZipFile('/content/drive/MyDrive/to.zip')
    to_dir = to_zip.extractall()

    # Load input and target image paths
    ti_paths = sorted(glob.glob('/content/ti/*.jpg'))
    to_paths = sorted(glob.glob('/content/to/*.jpg'))

    # Load and resize input and target images
    inputs = []
    targets = []
    for ti_path, to_path in zip(ti_paths, to_paths):
        ti_image = cv2.imread(ti_path, cv2.IMREAD_COLOR)
        to_image = cv2.imread(to_path, cv2.IMREAD_COLOR)

        # Check if images are valid
        if ti_image is None or to_image is None:
            continue

        ti_image = cv2.resize(ti_image, (256, 256))
        to_image = cv2.resize(to_image, (256, 256))
        inputs.append(ti_image)
        targets.append(to_image)

    # Convert input and target images to arrays and normalize pixel values to range [-1, 1]
    inputs = np.array(inputs) / 127.5 - 1.
    targets = np.array(targets) / 127.5 - 1.

    # Split dataset into training and testing sets
    train_inputs = inputs[:5000]
    train_targets = targets[:5000]
    test_inputs = inputs[5000:]
    test_targets = targets[5000:]

    # Create Tensorflow Dataset
    train_dataset = tf.data.Dataset.from_tensor_slices((train_inputs, train_targets)).shuffle(5000).batch(BATCH_SIZE)
    test_dataset = tf.data.Dataset.from_tensor_slices((test_inputs, test_targets)).batch(BATCH_SIZE)

    return train_dataset, test_dataset


In [ ]:
train_dataset, test_dataset = create_dataset()
for epoch in range(EPOCHS):
    print('Epoch', epoch+1, 'of', EPOCHS)
for input_image, target in train_dataset:
    gen_loss, disc_loss = train_step(input_image, target)
    print('Generator loss:', gen_loss.numpy(), 'Discriminator loss:', disc_loss.numpy())

    train_dataset, test_dataset = create_dataset()

for epoch in range(EPOCHS):
    print('Epoch', epoch+1, 'of', EPOCHS)
    for input_image, target in train_dataset:
        gen_loss, disc_loss = train_step(input_image, target)
        print('Generator loss:', gen_loss.numpy(), 'Discriminator loss:', disc_loss.numpy())

